In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
import pandas as pd
import json
import configparser
from pathlib import Path
import os

def tratamento_de_produtos(df: 'DataFrame', objeto: str) -> None:
    """ Esta """

### Para a previsão funcionar é necessário que o input dos dados esteja com um a seguinte estrutura:
### Arquivo CSV, 1 Coluna com o a timestamp e uma coluna com o nome 'preco' com os valores
### É necessário que a primeira linha do arquivo seja o header dele.

### Também é necessário que o arquivo 'configuracoes.conf' esteja na mesma pasta que este notebook
configs = configparser.ConfigParser()
configs.read("configuracoes.conf",encoding='utf-8')

data_folder = Path(configs["DEFAULT"]["folder_path"])
path = data_folder / configs["DEFAULT"]["file"]
qtd_Meses = int(configs["DEFAULT"]["meses"])
objeto = configs["DEFAULT"]["objeto"]
regiao = configs["DEFAULT"]["regiao"]

In [2]:
#monta a base de dados para treinar a rede
cesta_total = pd.read_excel(path) # lê os dados
cesta_total = cesta_total.iloc[:-qtd_Meses:]
cesta_total.drop(['salario', 'razao', 'ano'], axis=1, inplace=True)

demanda = cesta_total[['preco']] # pega a coluna de preco
n_steps = 12 # define a quantidade de meses que serão utilizados para prever o mês seguinte

# cria n_steps colunas com a demanda defasada. 
for time_step in range(1, n_steps+1):
    demanda['preco'+str(time_step)] = demanda[['preco']].shift(-time_step).values

demanda.dropna(inplace=True) # deleta linhas com valores nulos
demanda.reset_index(drop=True, inplace=True)
demanda = demanda / 1000

#Normalizando dividindo por 1000
#for attribute in demanda.columns.to_list():
    #demanda[attribute] = demanda[attribute]/1000

def model_dnn():
  model=Sequential()
  model.add(tf.keras.layers.SimpleRNN(32, input_shape=(12,1),return_sequences=True, stateful=True,batch_input_shape = (1,12,1),activation="tanh"))
  model.add(tf.keras.layers.SimpleRNN(32,stateful=True))
  model.add(Dense(1))
  model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003),metrics = ['mse', 'mae', tf.keras.metrics.RootMeanSquaredError()])
  return model

##Prevendo-
ate_ultimoMes = demanda.shape[0] #Dá o valor do último mês
input_limit = ate_ultimoMes + 12 #define o limite superior da série de input
n_steps = 12 # define a quantidade de períodos de tempo 
meses = qtd_Meses #de x em x meses
yhat = []

for i in range(0,meses):
    #Cria um novo training set para treinar o modelo para o mês seguinte com os valores preditos nos mêses anteriores
    cesta_training = cesta_total[['preco']]
    cesta_training = pd.concat([cesta_training, pd.DataFrame({'preco':yhat})], axis=0)
    cesta_training.reset_index(drop=True,inplace=True)
    demandanova = cesta_training[['preco']] # pega a coluna de preco

    # cria n_steps colunas com a demanda defasada. 
    for time_step in range(1, n_steps+1):
        demandanova['preco'+str(time_step)] = demandanova[['preco']].shift(-time_step).values

    demandanova.dropna(inplace=True) # deleta linhas com valores nulos
    demandanova.reset_index(drop=True, inplace=True)
    demandanova = demandanova / 1000

    #Coletando o training set
    X = demandanova.iloc[:ate_ultimoMes, :n_steps].values
    X = X.reshape((X.shape[0], X.shape[1], 1))

    #Coletando as labels
    Y = demandanova.iloc[:ate_ultimoMes, 12:].values

    #instanciando e treinando os modelos
    model_eightMonths = model_dnn()
    model_eightMonths.fit(X,Y, epochs=123, batch_size=1,
                   verbose=1,
                  shuffle=False)#,
                  #validation_split=0.2 )

    #Coleta o e organiza o input
    x_input = cesta_training['preco'].iloc[ate_ultimoMes:input_limit].values
    x_input = x_input/1000
    x_input = x_input.reshape((1, 12, 1))
    stimation = model_eightMonths.predict(x_input)
    yhat.append(stimation[0][0])

    #incrementando para o próximo mês
    ate_ultimoMes = ate_ultimoMes + 1
    input_limit = input_limit + 1

print("Fim da previsão")
resultados = [yhat[i] for i in range(0,len(yhat))]
objeto = objeto.replace(" ",'_')
output = {objeto.lower():str(resultados)}
filename = f"previsao_RNNRecursivo_{objeto.lower()}_{regiao.lower()}_{qtd_Meses}_meses.json"
pasta_atual = os.getcwd()
full_url = Path(pasta_atual) / 'resultados' / 'previsoes' / filename
os.makedirs(os.path.dirname(full_url), exist_ok=True)
with open(full_url,"w") as file:
    json.dump(output,file,ensure_ascii=False)
print("Arquivo com resultados gerados")

Epoch 1/123
189/189 [==============================] - 1s 3ms/step - loss: 0.0109 - mse: 0.0109 - mae: 0.0548 - root_mean_squared_error: 0.1044
Epoch 2/123
189/189 [==============================] - 1s 3ms/step - loss: 0.0015 - mse: 0.0015 - mae: 0.0301 - root_mean_squared_error: 0.0387
Epoch 3/123
189/189 [==============================] - 1s 3ms/step - loss: 0.0020 - mse: 0.0020 - mae: 0.0329 - root_mean_squared_error: 0.0447
Epoch 4/123
189/189 [==============================] - 1s 3ms/step - loss: 0.0025 - mse: 0.0025 - mae: 0.0345 - root_mean_squared_error: 0.0497
Epoch 5/123
189/189 [==============================] - 0s 3ms/step - loss: 0.0029 - mse: 0.0029 - mae: 0.0354 - root_mean_squared_error: 0.0539
Epoch 6/123
189/189 [==============================] - 1s 3ms/step - loss: 0.0034 - mse: 0.0034 - mae: 0.0364 - root_mean_squared_error: 0.0583
Epoch 7/123
189/189 [==============================] - 1s 3ms/step - loss: 0.0039 - mse: 0.0039 - mae: 0.0389 - root_mean_squared_error: